<a href="https://colab.research.google.com/github/hmaldonado12/7506R-1C2024-GRUPO6/blob/main/tp2/7506R_TP2_GRUPO06_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


In [ ]:
df_train_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df_test_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

df_train = df_train_original.copy()
df_test = df_test_original.copy()

# Preprocesamiento del dataset.

En esta notebook nos encargamos de efectuar la limpieza de dataset, la detección de datos incorrectos y/o vacíos, outliers, todo aquello que pueda imposibilitarnos a usar los modelos propuestos.

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           50000 non-null  int64 
 1   review_es    50000 non-null  object
 2   sentimiento  50000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


Podemos ver que el dataset train no tiene datos faltantes.

In [ ]:
df_train[df_train.duplicated(keep=False)]

,ID,review_es,sentimiento


Tampoco tiene filas duplicadas.

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599 entries, 0 to 8598
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         8599 non-null   int64 
 1   review_es  8599 non-null   object
dtypes: int64(1), object(1)
memory usage: 134.5+ KB


In [ ]:
df_test[df_test.duplicated(keep=False)]

,ID,review_es


El mismo resultado para test.

# Bag of words

# Naive Bayes
Es un método de aprendizaje supervisado basado en la aplicación del teorema de Bayes con el supuesto "ingenuo" de independencia condicional entre cada par de características dado el valor de la variable de clase.
Según bayes si tengo un conjunto de clase C y para un documento D, si encuentro una clase que maximice su probabilidad condicional, entonces el documento pertenece a esa clase.

## Objetivo
- Dado un determinado numero de documentos de criticas sobre películas, determinar para cada documento si la crítica es positiva o negativa

## Descripción de los datos
- **ID:** Identificador unico de cada review
- **review_es:** Documento que contiene la crítica
- **sentimiento:** Clases la cual pertenece una crítica la misma puede ser POSITIVA o NEGATIVA

In [3]:
cinema_review = pd.read_csv('train.csv')
cinema_review

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
49995,49995,Pensé que esta película hizo un buen trabajo a...,positivo
49996,49996,"Mala parcela, mal diálogo, mala actuación, dir...",negativo
49997,49997,Soy católica enseñada en escuelas primarias pa...,negativo
49998,49998,Voy a tener que estar en desacuerdo con el com...,negativo


## Separación de datos
### Generación de conjunto de test y train

In [4]:
X = cinema_review.review_es
y = cinema_review.sentimiento
X_train_review, X_test_review, y_train_sentimiento, y_test_sentimiento = train_test_split(X,y,stratify=y,test_size=0.20,random_state=42)

#### Textos de entranamiento

In [5]:
X_train_review

47808    Cogí esta pequeña joya totalmente por accident...
20154    No puedo creer que me permita a esta película ...
43069    * ¡Alerta de spoiler! * Solo me pone el nervio...
19413    Si hay una cosa que he aprendido de ver el crí...
13673    Recuerdo cuando esto estaba en los teatros, di...
                               ...                        
31092    Es la década de 1920.Y un hombre llamado Walt ...
22917    El primer (y único) tiempo que vi "Sombras" fu...
47481    Esta fue una pérdida de tiempo.PELIGRO: Si lo ...
35597    Esto es, con mucho, el cine indio de la pelícu...
27491    Esta película para siempre dejó una impresión ...
Name: review_es, Length: 40000, dtype: object

In [6]:
y_train_sentimiento

47808    positivo
20154    negativo
43069    negativo
19413    negativo
13673    negativo
           ...   
31092    positivo
22917    negativo
47481    negativo
35597    negativo
27491    negativo
Name: sentimiento, Length: 40000, dtype: object

**Antes de comenzar a realizar las predicciones es necesario realizar las feature engineer, que permitan sacar el ruido que puedan tener los datos y que esto no influya en el modelo que clasifica las críticas**

## Feature Engineer
### Normalización
Para cada texto del dataFrame vamos a extraer todos los caracteres distintos a letras y números, para disminuir la introducción de ruidos al modelo de NaiveBayes

In [7]:
for indice in range(0, len(X_train_review)):
    X_train_review.iloc[indice] = unicodedata.normalize("NFKD", X_train_review.iloc[indice]).encode("ascii","ignore").decode("ascii")
X_train_review

47808    Cogi esta pequena joya totalmente por accident...
20154    No puedo creer que me permita a esta pelicula ...
43069    * Alerta de spoiler! * Solo me pone el nervio ...
19413    Si hay una cosa que he aprendido de ver el cri...
13673    Recuerdo cuando esto estaba en los teatros, di...
                               ...                        
31092    Es la decada de 1920.Y un hombre llamado Walt ...
22917    El primer (y unico) tiempo que vi "Sombras" fu...
47481    Esta fue una perdida de tiempo.PELIGRO: Si lo ...
35597    Esto es, con mucho, el cine indio de la pelicu...
27491    Esta pelicula para siempre dejo una impresion ...
Name: review_es, Length: 40000, dtype: object

## Tokenizar y vectorizar documentos para el entrenamiento del modelo

In [8]:
count_vect = CountVectorizer(
    min_df=30,
    ngram_range=(1,2),
    analyzer='word'
)
X_train_review_count = count_vect.fit_transform(X_train_review)
X_train_review_count.shape

(40000, 43829)

In [9]:
tfidf_transformer = TfidfTransformer()
X_train_review_tfidf = tfidf_transformer.fit_transform(X_train_review_count)
X_train_review_tfidf .shape

(40000, 43829)

## Entrenar clasificador

In [10]:
clf_naive_bayes = MultinomialNB(alpha=1).fit(X_train_review_tfidf, y_train_sentimiento)

## Realizar predicción con el conjunto de entrenamiento

In [11]:
X_test_review_count = count_vect.transform(X_test_review)
X_test_review_tfidf = tfidf_transformer.fit_transform(X_test_review_count)
predicted = clf_naive_bayes.predict(X_test_review_tfidf)
predicted

array(['negativo', 'positivo', 'positivo', ..., 'negativo', 'positivo',
       'positivo'], dtype='<U8')

## Evaluación del Modelo

In [12]:
print(metrics.classification_report(y_test_sentimiento, predicted))

              precision    recall  f1-score   support

    negativo       0.87      0.84      0.85      5000
    positivo       0.84      0.87      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000


## Cross-validation

In [13]:
clf_naive_bayes = MultinomialNB(alpha=1).fit(X_train_review_tfidf, y_train_sentimiento)
scores_cv_nb = cross_val_score(clf_naive_bayes,
                               X_train_review_tfidf, 
                               y_train_sentimiento,
                               cv = 10, 
                               scoring='f1_macro'
)
print("f-score=",round(scores_cv_nb.mean(),4))

f-score= 0.8621


# Predicción con los datos de test.csv

In [14]:
test = pd.read_csv('test.csv')
X_test_counts_cinema = count_vect.transform(test.review_es)
X_new_tfidf_cinema_test = tfidf_transformer.transform(X_test_counts_cinema)

predicted_cinema = clf_naive_bayes.predict(X_new_tfidf_cinema_test)
predicted_cinema

array(['negativo', 'negativo', 'negativo', ..., 'positivo', 'negativo',
       'negativo'], dtype='<U8')

# Exportar datos para kaggle

In [15]:
csv_nuevo_solution = pd.DataFrame(predicted_cinema)
csv_nuevo_solution["ID"] = np.arange(60000,(8599+60000),1)
csv_nuevo_solution.columns = ['sentimiento', 'ID']
csv_nuevo = csv_nuevo_solution.reindex(['ID', 'sentimiento'], axis=1)
csv_nuevo.to_csv('solution_naive_bayes_final.csv',sep=',', index=False)